# Проект "Приложение: Сервис для чтения книг по подписке".

## Цели исследования: 

- проанализировать базу данных о книгах, издательствах и авторах;
- сформулировать ценностное предложение для нового продукта.

### Задачи исследования:
- Посчитать, сколько книг вышло после 1 января 2000 года;
- Посчитать для каждой книги количество обзоров и среднюю оценку;
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц;
- Определить автора с самой высокой средней оценкой книг — учитывать только книги с 50 и более оценками;
- Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

### Описание данных:

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

## Загрузка данных:

In [1]:
!pip install psycopg2-binary

In [2]:
!pip install sqlalchemy

In [3]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [4]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных

In [5]:
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db'],
) 

In [6]:
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [7]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


## Исследуем таблицы и выведем первые строки.

In [8]:
def show(sql):
    return pd.io.sql.read_sql(sql, con = engine)

In [9]:
query_books = '''
        SELECT * FROM books
        '''
books = show(query_books)

query_authors = '''
                SELECT * FROM authors
                '''
authors = show(query_authors)

query_publishers = '''
                   SELECT * FROM publishers
                   '''
publishers = show(query_publishers)

query_ratings = '''
                SELECT * FROM ratings
                '''
ratings = show(query_ratings)

query_reviews = '''
                SELECT * FROM reviews
                '''
reviews = show(query_reviews)

In [10]:
#Таблица книги просмотр:
books.head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [11]:
#таблица автор просмотр:
authors.head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [12]:
#Таблица издательство просмотр:
publishers.head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [13]:
#Таблица рейтинги просмотр:
ratings.head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [14]:
#таблица reviews просмотр:
reviews.head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Посчитаем, сколько книг вышло после 1 января 2000 года.

создадим функцию для формирования таблиц:

In [15]:
def read(sql):
    return pd.io.sql.read_sql(sql, con = engine) 

Получается задействована таблица:"books", так как в ней имеются публикации книжек после 01 января 2020 года.

In [16]:
books = '''
        SELECT COUNT(DISTINCT book_id) as books_published
        FROM books
        WHERE publication_date > '2000-01-01'
        '''
read(books)

,books_published
0,819


### Вывод:
Книг, опубликованных после 01.01.2020 года - 819.    

## Посчитаем для каждой книги количество обзоров и среднюю оценку.

Чтобы посчитать для каждой книги количество обзоров нам потребуется таблица review (для подсчета обзоров) и таблица rating для нахождения средней оценки, и также название книги - основная таблица title.

In [17]:
review = '''
        SELECT b.book_id AS book_id,
               b.title AS title,
               COUNT(DISTINCT r.review_id) AS reviews,
               ROUND(AVG(ra.rating),0) AS avg_rating
               
        FROM books as b
        INNER JOIN ratings ra ON ra.book_id = b.book_id
        LEFT JOIN reviews r ON r.book_id = b.book_id
                     
        GROUP BY b.book_id, b.title
        ORDER BY reviews DESC
        
        '''
read(review)

,book_id,title,reviews,avg_rating
0,948,Twilight (Twilight #1),7,4.0
1,963,Water for Elephants,6,4.0
2,734,The Glass Castle,6,4.0
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.0
4,695,The Curious Incident of the Dog in the Night-Time,6,4.0
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,4.0
996,808,The Natural Way to Draw,0,3.0
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.0
998,221,Essential Tales and Poems,0,4.0


### Вывод:
Больше всего review - у книге "Twilight"- Сумерки. Обзоров - 7 и средняя оценка - 4. Всего книг 1000.

## Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры.

Чтобы определить издательство, нам нужно будет присоединить основную таблицу "книг" к таблице "издательство".

In [18]:
publisher = '''
            SELECT p.publisher, 
                   COUNT(b.book_id) as books
            FROM publishers p
            JOIN books b ON p.publisher_id=b.publisher_id
            WHERE b.num_pages > 50
            GROUP BY p.publisher_id
            ORDER BY COUNT(book_id) DESC
            LIMIT 5
            '''
read(publisher)    

,publisher,books
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Bantam,19


Мы определили топ-5 издательств, которые выпустили максимальное количество книг со страницами больше 50.

### Вывод:
Издательство которое выпустило наибольшее число книг толще 50 страниц: 'Penquin Books'. Видимо это иностранный сервис по подписке книг.Потому что, я пользовалась издательством Penguin Books, которое печатало книги на английском языке.

## Определим автора с самой высокой средней оценкой книг — учитываем только книги с 50 и более оценками.

Нам надо найти автора из таблицы "authors" , среднюю оценку книг мы можем посмотреть в таблице "ratings", но и книги мы смотрим в нашей основной таблице "books":

In [19]:
author =''' WITH avg AS (SELECT b.book_id,
                        ROUND(AVG(rating),2) AS avg_rating
                        FROM books b
                        JOIN ratings r ON b.book_id=r.book_id
                        JOIN authors a ON b.author_id=a.author_id
                        GROUP BY b.book_id
                        HAVING COUNT(r.rating_id) >= 50
                        ORDER BY (b.book_id) DESC)
                SELECT a.author,
                ROUND(AVG(avg.avg_rating),2) AS avg_rating
                ---COUNT(b.book_id) as books
                FROM authors a
                JOIN books b ON a.author_id=b.author_id
                RIGHT JOIN avg ON b.book_id=avg.book_id
                GROUP BY a.author
                ORDER BY avg_rating DESC
                LIMIT 5'''
read(author)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.29
1,Markus Zusak/Cao Xuân Việt Khương,4.26
2,J.R.R. Tolkien,4.26
3,Louisa May Alcott,4.19
4,Rick Riordan,4.08


### Вывод:
Самый популярный автор - Джоан Роулинг, которая написала Гарри Поттера и вместе с Мари ГрандПре - это иллюстратор книг.

## Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

С помощью агрегирующих функций, посчитаем количество обзоров "reviews" и  рейтинг "ratings" больше 48 оценок и затем посчитаем среднее количество обзоров от пользователей: путем деления суммы обзоров на количество пользователей.

In [20]:
users_review = '''
               WITH users AS (SELECT  username,
                              COUNT(rating_id) AS ratings_count
                              FROM ratings
                              GROUP BY username
                              HAVING COUNT(rating_id) > 48),
               reviews as (SELECT username,
                            COUNT(review_id) as reviews
                            FROM reviews
                            GROUP BY username)
               SELECT ROUND(SUM(r.reviews)/COUNT(u.username),0) AS avg_review_count
                      FROM users u
                      JOIN reviews r ON u.username=r.username                         
                      '''
read(users_review)

,avg_review_count
0,24.0


### Вывод:
Среднее количество обзоров от пользователей, которые поставили больше 48 оценок - 24.

## Общий вывод по исследованиям:
- Количество книг, которое вышло после 1 января 2000 года - 819. Всего в исследовании учавствовало - 1000 строк id_book. Следовательно это приложение начало свою работу совсем недавно;
- Топ-5 книг с высоким средним рейтингом:
    - Twilight (сумерки) - рейтинг 7
    - Water for Elephants (воды слонам) - рейтинг 6
    - The Glass Castle (стеклянный замок) -  рейтинг 6
    - Harry Potter and the Prisoner of Azkaban (Гарри Потер и узник Аскабана) - рейтинг 6
    - The Curious Incident of the Dog in the Night-Time (Загадочное ночное убийство собаки) - рейтинг 6
- Издательство которое выпустило наибольшее число книг толще 50 страниц: 'Penquin Books'. Видимо это иностранный сервис по   подписке книг.Потому что, я пользовалась издательством Penguin Books, которое печатало книги на английском языке;
- Самый популярный автор - Джоан Роулинг, которая написала Гарри Поттера и вместе с Мари ГрандПре - это иллюстратор книг.
  Если поставить лимит пять: то топ-пять авторов это:
  - 1 - J.K. Rowling/Mary GrandPré (Джоан Роулинг) - средняя оценка 4.29 
  - 2 - Markus Zusak/Cao Xuân Việt Khương (Маркус Зюзяк) - средняя оценка	4.26 
  - 3 -J.R.R. Tolkien (Толкен) - средняя оценка 4.26 
  - 4 - Louisa May Alcott (Луиза Мэй Элькот) - средняя оценка 4.19 
  - 5 - Rick Riordan (Рик Риордан) - средняя оценка	4.08;

- Среднее количество обзоров от пользователей, которые поставили больше 48 оценок - их 24. Посещаемость приложения не очень высокая, что говорит о том, что только недавно стало использоваться пользователями.

### Рекомендации:
- расширить библиотеку книг;
- и задействовать работу маркетинговой службы над увеличением числа посещаемости приложения и тем самым больше будет оценок от пользователей.

Спасибо большое!